1- Behavioural plots for all conditions.

2- Unimanual: comparing RT, MT, MD, between different directions and Right and Left hands.

3- Bimanual: comparing RT, MT, MD of the same reach across matched, mirror, mirror-diagonal, unrelated conditions.